In [1]:
from fractions import Fraction
from pathlib import Path
import pandas as pd
import numpy as np
import pyarrow
import yaml

# Suspicion Score

There are three reasons why a node might be suspicious: 
1. The KYC data (node data, eg. occupation) is suspicious
2. The transactions (edges, e.g. e-transfer messages) around that node are suspicious
3. The graph structure around that node is suspicious (e.g. in a 'mule' structure). 

The purpose of this notebook is to aggregate those three factors (and the indicators contained therein) to come up with an explainable total node suspicion score. Because graph structures are complicated and composed of international & external customers - (of which we only have a small fraction), we only focus on the first two reasons, and assume that suspicious graph structures will emerge downstream during pagerank.

If you want to edit weights. Edit the weights.yml file in the current folder.

In [2]:
WEIGHTPATH = Path('./weights.yml')

with open(WEIGHTPATH, 'r') as file: 
    weights = yaml.safe_load(file)
    
weights

{'NODE_WEIGHTS': {'LABEL': 1,
  'OCC_INT': '1/6',
  'OCC_WEALTH': '1/6',
  'OCC_ANIMAL': '1/6',
  'C_LARGE': '1/2'},
 'EDGE_WEIGHTS': {'E_AT_RISK': 1,
  'E_ROLE': 1,
  'E_TRAD_MED': 1,
  'W_TO_COUNTRY': '1/3',
  'W_FROM_COUNTRY': '1/3',
  'W_EXTERNAL_TO_ANIMAL': '1/3',
  'T_ANIMAL': '1/4',
  'T_ANIMAL_LARGE': '1/4',
  'T_INT': '1/4',
  'T_TO_SHIPPING': '1/4'}}

# Node Suspicion Score

In [3]:
DATAPATH = Path('../data/processed')
KYCPATH = DATAPATH / 'kyc.parquet'
CASHPATH = DATAPATH / 'cash.parquet'

node_weights = {w: Fraction(v) for w, v in weights['NODE_WEIGHTS'].items()}

## KYC Aggregation
We first combine the KYC features `label`, `occ_int`, `occ_wealth`, `occ_animal` into one one score using the weights in `weights.yml`

In [4]:
def kyc_agg(r, node_weights): 
    occ_agg = \
        node_weights['LABEL']*r.label +\
        node_weights['OCC_INT']*r.occ_int +\
        node_weights['OCC_WEALTH']*r.occ_wealth +\
        node_weights['OCC_ANIMAL']*r.occ_animal

    return float(occ_agg)

kyc_df = pd.read_parquet(KYCPATH)

kyc_df['kyc_agg'] = kyc_df.apply(lambda r: kyc_agg(r, node_weights), axis=1)
kyc_df.sample(3)

,name,gender,occupation,age,tenure,cust_id,occ_wealth,occ_animal,occ_int,label,kyc_agg
64325,YASHVI DE,male,Kinesiologist,23.0,5.0,CUST62507742,0,0,0,0,0.000000
27464,JOSEPH ZHANG,male,School Teacher,37.0,11.0,CUST22037943,0,0,0,0,0.000000
91910,FRANCISCO KAISER,male,Import/Export Business Owner,48.0,10.0,CUST15760108,1,0,1,1,1.333333


## Cash Aggregation
For the purposes of this project, we consider cash transactions as only occuring between one customer (i.e. the bank does *not* count as a customer). This prevents graph neighbourhoods from being too dense (everyone is connected to the bank, so everyone is a 2-hop neighbour). 

For practical purposes, this means that indicators related to deposit/withdrawal size become node scores, rather than edge scores.

There is only one cash feature currently, `c_large`. `cash_agg` is therefore defined as the proportion of a customers cash transactions that have been defined as being unusually large.

In [5]:
cash_df = pd.read_parquet(CASHPATH)
cash_df.sample(3)

,cust_id,trxn_amount,type,trxn_id,occ_wealth,occ_animal,occ_int,label,occupation,c_large
195253,CUST89177216,7805,deposit,FUBR75522255,1,0,0,0,Pawn Shop Owner,0
100484,CUST41736633,14435,deposit,LEFQ87126344,1,0,1,0,Currency Exchange Owner,1
129532,CUST50849330,395,withdrawal,ZDCF55676186,0,0,0,0,School Bus Driver,0


In [6]:
cash_df = pd.read_parquet(CASHPATH)
cash_agg_df = cash_df.groupby('cust_id')['c_large'].mean().reset_index()
cash_agg_df = cash_agg_df.rename(columns={'c_large':'c_agg'})
cash_agg_df.sample(3)

,cust_id,c_agg
1347,CUST12014674,0.0
30617,CUST55499122,0.0
43304,CUST74054228,0.0


## Node Score Calculation
We assume that the total node suspicion score is an average of the aggregate kyc suspicion score  and the aggregate cash suspicion score above.

In [7]:
#Join KYC and Cash Data
merged = kyc_df.merge(cash_agg_df, on='cust_id', how='left')
merged['c_agg'] = merged['c_agg'].fillna(0)

#Aggregate 
node_df = merged[['cust_id', 'kyc_agg', 'c_agg']].copy()
node_df['score'] = (node_df['kyc_agg'] + node_weights['C_LARGE']*node_df['c_agg'])

#Clean
node_df = node_df[['cust_id', 'score']]

#Export
node_df.to_parquet(DATAPATH/ 'pagerank' / 'input_node_scores.parquet', index=False)
node_df.sort_values('score', ascending=False)

,cust_id,score
155546,CUST43500432,1.833333
68589,CUST31965538,1.666667
57930,CUST27210892,1.666667
164188,CUST40682551,1.666667
140389,CUST69789020,1.666667
...,...,...
73748,CUST49137654,0.0
73749,CUST76627038,0.0
73750,CUST81879435,0.0
73751,CUST28203470,0.0


# Edge Suspicion Score
Features starting with **agg** are aggregates of pre-computed transaction features.

In [8]:
DATAPATH = Path('../data/processed')
EPATH = DATAPATH / 'emt.parquet'
WIREPATH = DATAPATH / 'wire.parquet'

edge_weights = {w: Fraction(v) for w, v in weights['EDGE_WEIGHTS'].items()}

## E-transfer aggregation

In [9]:
def e_transfer_agg(r, edge_weights): 
    e_agg = \
        edge_weights['E_ROLE']*r.e_role +\
        edge_weights['E_AT_RISK']*r.e_at_risk +\
        edge_weights['E_TRAD_MED']*r.e_trad_med

    return float(e_agg)

edf = pd.read_parquet(EPATH)
edf['e_agg'] = edf.apply(lambda r: e_transfer_agg(r, edge_weights), axis=1)
edf.sample(3)

,cust_id_sender,cust_id_receiver,name_sender,name_receiver,trxn_message,emt_value,trxn_id,regex_flag,occ_wealth_receiver,occ_animal_receiver,...,e_trad_med,trxn_type,t_to_animal,t_from_animal,t_to_animal_large,t_from_animal_large,t_to_int,t_from_int,t_to_shipping,e_agg
31735,CUST72112021,CUST77079593,THOMAS ROY,ANVI JHAVERI,,490.0,CPNR54456127,0,0.0,0.0,...,0,emt,0,0,0,0,0,0,0,0.0
344101,CUST41525073,CUST92243631,ISABELA ARTURO SANABRIA DEL VALLE,CHRISTINA THOMPSON,,800.0,IMAO74149170,0,0.0,0.0,...,0,emt,0,0,0,0,0,0,0,0.0
447662,CUST34622355,CUST86163504,BRIAN MOLINA,SAMANTHA DOMINGUEZ,,610.0,ORCW15154857,0,0.0,0.0,...,0,emt,0,0,0,0,0,0,0,0.0


## Wire transfer aggregation
The aggregate wire transfer score is an average of the wire transfer indicators `w_to_country`, `w_from_country`, `w_external_to_animal`.

In [10]:
def wire_transfer_agg(r, edge_weights):
    w_agg = \
        edge_weights['W_TO_COUNTRY']*r.w_to_country +\
        edge_weights['W_FROM_COUNTRY']*r.w_from_country +\
        edge_weights['W_EXTERNAL_TO_ANIMAL']*r.w_external_to_animal
    
    return float(w_agg)

wdf = pd.read_parquet(WIREPATH)
wdf['w_agg'] = wdf.apply(lambda r: wire_transfer_agg(r, edge_weights), axis=1)
wdf.sample(3)

,cust_id_sender,cust_id_receiver,name_sender,name_receiver,trxn_value,country_sender,country_receiver,trxn_id,occ_wealth_receiver,occ_animal_receiver,...,w_external_to_animal,trxn_type,t_to_animal,t_from_animal,t_to_animal_large,t_from_animal_large,t_to_int,t_from_int,t_to_shipping,w_agg
40962,CUST29016088,CUST27984649,WILLIAM DUFRESNE,ZHOU YU MEI,7115.0,CA,CA,PXIJ48540157,0.0,0.0,...,0,wire,0,0,0,0,0,1,0,0.0
6204,CUST89838708,EXTERNAL521215,DEREK LEE,ALEXIS VARGAS,1810.0,CA,CA,OIDM38434235,NaN,NaN,...,0,wire,0,0,0,0,0,0,0,0.0
47288,EXTERNAL897142,CUST64478481,LISA JAMES,MINERVA ARMENTA,7118.0,CA,CA,FAPW11964538,1.0,0.0,...,0,wire,0,0,0,0,0,0,0,0.0


## General Score Calculation

In [11]:
def compute_score(r, edge_weights, receiver=False):
    score = \
        edge_weights['T_INT']*r.iloc[2] +\
        edge_weights['T_ANIMAL']*r.iloc[1] +\
        edge_weights['T_ANIMAL_LARGE']*r.iloc[0]
    
    if receiver:
        score += edge_weights['T_TO_SHIPPING']*r.iloc[3]
    
    return float(score)

In [12]:
#Col names
sender_cols = ['t_from_animal', 't_from_animal_large', 't_from_int']
receiver_cols = ['t_to_animal', 't_to_animal_large', 't_to_int', 't_to_shipping']

#***E-TRANSFER**
edf['score_sender'] = edf[sender_cols].apply(compute_score, axis=1, args=(edge_weights, False))
edf['score_receiver'] = edf[receiver_cols].apply(compute_score, axis=1, args=(edge_weights, True))

#add score_sender and score_receiver and normalize the score to be between 0 and 1
edf['g_agg'] = (edf['score_sender'] + edf['score_receiver'])
edf['g_agg'] = (edf['g_agg'] - edf['g_agg'].min())/(edf['g_agg'].max() - edf['g_agg'].min())
edf.drop(columns=['score_sender', 'score_receiver'], inplace=True)

#**WIRE-TRANSFER
wdf['score_sender'] = wdf[sender_cols].apply(compute_score, axis=1, args=(edge_weights, False))
wdf['score_receiver'] = wdf[receiver_cols].apply(compute_score, axis=1, args=(edge_weights, True))

#add score_sender and score_receiver and normalize the score to be between 0 and 1
wdf['g_agg'] = (wdf['score_sender'] + wdf['score_receiver'])
wdf['g_agg'] = (wdf['g_agg'] - wdf['g_agg'].min())/(wdf['g_agg'].max() - wdf['g_agg'].min())
wdf.drop(columns=['score_sender', 'score_receiver'], inplace=True)

## Edge Score Calculation

In [13]:
edf['score'] = edf['g_agg'] + edf['e_agg']
wdf['score'] = wdf['g_agg'] + wdf['w_agg']

#exporting
edf.to_parquet(EPATH, index=False)
wdf.to_parquet(WIREPATH, index=False)

s1 = edf[['cust_id_sender', 'cust_id_receiver', 'score']].copy()
s2 = wdf[['cust_id_sender', 'cust_id_receiver', 'score']].copy()
s1['type'] = 'e'
s2['type'] = 'w'

pd.concat([s1,s2]).to_parquet(DATAPATH / 'pagerank' / 'input_edge_scores.parquet')